In [99]:
import os
import io
import base64
import json
from dotenv import load_dotenv
from PIL import Image
from typing import List, Optional
from pydantic import BaseModel, Field, conlist
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.output_parsers import PydanticOutputParser
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")
if not API_KEY:
    pass 
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro", 
    google_api_key=API_KEY
)
SuggestionType = str 
class MainSuggestions(BaseModel):
    """Exactly 4 suggestions, one for each main category."""
    movie_style_suggestion: SuggestionType = Field(
        ..., 
        description="A single, natural language suggestion for a specific movie or cinematic aesthetic (e.g., 'Give it the look of a 70s Western film')."
    )
    mood_suggestion: SuggestionType = Field(
        ..., 
        description="A single, natural language suggestion targeting a specific emotional mood (e.g., 'Inject a strong sense of serenity and calm')."
    )
    color_focus_suggestion: SuggestionType = Field(
        ..., 
        description="A single, natural language suggestion focusing on a specific color or palette (e.g., 'Emphasize the deep, cool blue tones throughout the image')."
    )
    other_main_suggestion: SuggestionType = Field(
        ..., 
        description="A single, natural language, high-level suggestion not covered by the other categories (e.g., 'Apply a classic portrait finish')."
    )

class CombinedSuggestions(BaseModel):
    """Contains the 4 main suggestions and the 15 general suggestions."""
    main_suggestions: MainSuggestions = Field(
        ...,
        description="Exactly 4 high-level, categorized suggestions."
    )
    normal_suggestions: conlist(SuggestionType, min_length=15, max_length=15) = Field(
        ..., 
        description="Exactly 15 general, natural language enhancement suggestions."
    )
parser = PydanticOutputParser(pydantic_object=CombinedSuggestions)
format_instructions = parser.get_format_instructions()

def pil_to_data_uri(img, fmt="JPEG"):
    """Converts a PIL Image object to a base64 Data URI."""
    buf = io.BytesIO()
    img.save(buf, format=fmt, quality=90)
    return "data:image/jpeg;base64," + base64.b64encode(buf.getvalue()).decode()
def gemini_user_style_suggestions(image: Image.Image):
    """
    Analyzes the image and returns exactly 4 main and 15 normal human-style suggestions, structured in JSON.
    """
    prompt = (
        "Analyze the image carefully and return ONLY a single JSON object "
        "containing exactly 4 main suggestions only in 4 to 6 strictly words  (one for each category: Movie Style, Mood, Color Focus, Other) "
        "and exactly 15 general suggestions, all in natural human language.\n\n"

        "**STRICT FORMAT INSTRUCTIONS**:\n"
        + format_instructions +
        "\n\n"

        "**RULES FOR ALL 19 SUGGESTIONS**:\n"
        "- The output MUST contain exactly one `main_suggestions` object with 4 fields, and one `normal_suggestions` list with exactly 15 elements.\n"
        "- Keep each suggestion short and natural, as if a user wrote it.\n"
        "- NO numbers, NO technical terms, NO percentages, NO stops.\n"
        "- NO crop or composition instructions.\n"
        "- Suggestions MUST relate only to exposure, contrast, tone, temperature, tint, "
        "highlights, shadows, whites, blacks, saturation, vibrance, or general color feel.\n"
        "- main suggestions must be strictly 4 to 6 words each.\n"
        "- Style should match examples like:\n"
        "  • Main: 'Give this photo a dark, cinematic grade like a Christopher Nolan film.'\n"
        "  • Normal: 'Slightly reduce the overall exposure to add drama.'\n"
    )

    data_uri = pil_to_data_uri(image)

    msg = {
        "role": "user",
        "content": [
            {"type": "text", "text": prompt},
            {"type": "image_url", "image_url": data_uri}
        ]
    }
    resp = gemini.invoke([msg])
    raw = getattr(resp, "content", "")
    raw = raw.strip() if isinstance(raw, str) else str(resp).strip()

    if raw.startswith("```"):
        raw = raw.strip().strip("`").replace("json", "").strip()

    try:
        return parser.parse(raw)
    except Exception as e:

        print(f"Pydantic parsing failed: {e}. Attempting direct JSON load.")
        return CombinedSuggestions.model_validate(json.loads(raw))

In [100]:
import json
import os
from PIL import Image
OUT_PATH = "./light_suggestions.json"
IMAGE_PATH = "/home/logan78/Desktop/autocompletion_local/images/gal.jpg"

def collect_all_light_suggestions_from_model(image_path: str) -> dict:
    """
    Loads the image, calls gemini_user_style_suggestions(),
    flattens main + normal suggestions into a single list, and
    returns a dict with structured data.
    """
    img = Image.open(image_path).convert("RGB")
    result = gemini_user_style_suggestions(img)
    main = {
        "movie_style_suggestion": str(result.main_suggestions.movie_style_suggestion).strip(),
        "mood_suggestion": str(result.main_suggestions.mood_suggestion).strip(),
        "color_focus_suggestion": str(result.main_suggestions.color_focus_suggestion).strip(),
        "other_main_suggestion": str(result.main_suggestions.other_main_suggestion).strip(),
    }
    normal = [str(s).strip() for s in result.normal_suggestions]
    all_suggestions = [
        main["movie_style_suggestion"],
        main["mood_suggestion"],
        main["color_focus_suggestion"],
        main["other_main_suggestion"],
    ] + normal
    cleaned = [s for s in all_suggestions if isinstance(s, str) and s.strip()]
    if len(cleaned) != 19:
        print(f"Warning: expected 19 suggestions but got {len(cleaned)}. Saving whatever we have.")

    return {"main": main, "normal": normal, "all": cleaned}
try:
    data = collect_all_light_suggestions_from_model(IMAGE_PATH)
    with open(OUT_PATH, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"Saved suggestions to: {OUT_PATH}")
    print("\nPreview (first 8 suggestions):")
    for i, s in enumerate(data["all"][:8], start=1):
        print(f"{i}. {s}")
    print(f"\nTotal saved suggestions: {len(data['all'])}")
except FileNotFoundError:
    print(f"Image not found at {IMAGE_PATH}. Please update IMAGE_PATH and rerun.")
except Exception as e:
    print(f"Error while generating/saving suggestions: {e}")


Saved suggestions to: ./light_suggestions.json

Preview (first 8 suggestions):
1. Recreate a classic indie film aesthetic.
2. Create a warm and nostalgic mood.
3. Emphasize the warm golden hour tones.
4. Apply a soft and dreamy finish.
5. Increase the overall warmth of the image.
6. Gently lift the shadows on her face.
7. Add a bit more contrast for depth.
8. Enhance the golden glow in her hair.

Total saved suggestions: 19


In [105]:
import requests
from typing import List
def extract_ollama_response(text: str) -> str:
    """Extract plain response from Ollama JSON or raw text output."""
    try:
        import json
        data = json.loads(text)
        return data.get("response", text)
    except:
        return text

def strip_numbers(text: str) -> str:
    """Remove numbers from the generated prediction."""
    import re
    return re.sub(r"\d+", "", text)

OLLAMA_URL = "http://localhost:11434/api/generate"  
OLLAMA_MODEL = "llama3.2"

def autocomplete_lightart(base_sentence: str, light_suggestions: List[str]):
    prompt = f"""
You are an AUTOCOMPLETE assistant 

Use ONLY these light_suggestions:
{light_suggestions}

RULES:
- Continue the sentence EXACTLY from where it ends.
- do not change the base sentence.
- only use provided light_suggestions to autocomplete.
- ONLY natural color/tone language.
- only autocomplete by light_suggetions.

Complete: {base_sentence}
"""

    res = requests.post(
        OLLAMA_URL,
        json={
            "model": OLLAMA_MODEL,
            "prompt": prompt,
            "max_tokens": 500,
            "stream": False
        },
        timeout=20,
    )

    raw = extract_ollama_response(res.text.strip())
    raw = strip_numbers(raw)
    return raw


In [111]:
import json
import os
SAVED_PATH = "./light_suggestions.json"
BASE_SENTENCE = "Add a bit"
if not os.path.exists(SAVED_PATH):
    raise FileNotFoundError(f"Saved suggestions file not found: {SAVED_PATH}\nRun the Gemini cell first.")

with open(SAVED_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

all_suggestions = data.get("all") or []
if not all_suggestions:
    raise ValueError("No suggestions found in the saved file. Re-run Gemini to regenerate suggestions.")
all_suggestions = all_suggestions[:19]
try:
    completion = autocomplete_lightart(base_sentence=BASE_SENTENCE, light_suggestions=all_suggestions)
    print("\nOLLAMA AUTOCOMPLETE RESULT:\n", completion)
except Exception as e:
    print(f"Error while calling autocomplete_lightart: {e}")


OLLAMA AUTOCOMPLETE RESULT:
 ...more contrast for depth.
